In [1]:
import copy
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os
import numpy as np
import random
import yaml
from models.Flexible_DANN import Flexible_DANN
from PKLDataset import PKLDataset
from utils.general_train_and_test import general_test_model
from models.get_no_label_dataloader import get_target_loader

In [12]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def get_dataloaders(source_path, target_path, batch_size):
    """
        构建源域 (source) 和目标域 (target) 的 DataLoader。

        参数：
            source_path (str): 源域数据的 txt 文件路径。
                               每一行通常包含样本文件路径及其标签。
            target_path (str): 目标域数据的 txt 文件路径。
                               通常目标域没有标签
            batch_size (int): 每个 batch 的样本数量。

        返回：
            tuple:
                - source_loader : 源域的 DataLoader，
                  会返回 (x, y) 格式的批次数据。
                - target_loader : 目标域的 DataLoader，
                  会返回 x（无标签）。


        """

    source_dataset = PKLDataset(txt_path=source_path)
    source_loader = DataLoader(source_dataset, batch_size=batch_size, shuffle=True)
    target_loader = get_target_loader(target_path, batch_size=batch_size, shuffle=True)
    return source_loader, target_loader

def dann_lambda(epoch, num_epochs):
    """
    常用的 DANN λ 调度：从 0 平滑升到 0.6
    你也可以把 -10 调轻/重来改变上升速度
    """
    if epoch < 10:
        return 0.8
    elif epoch < 20:
        return 0.5
    else:
        return 0.3

def train_dann(model, source_loader, target_loader,
               optimizer, criterion_cls, criterion_domain,
               device, num_epochs=20, lambda_=dann_lambda,scheduler = None):
    best_gap = 0.5
    best_model_state = None
    patience = 0
    for epoch in range(num_epochs):
        cls_loss_sum, dom_loss_sum, total_loss_sum = 0.0, 0.0, 0.0
        total_cls_samples, total_dom_samples = 0, 0
        dom_correct, dom_total = 0, 0
        model.train()

        for (src_x, src_y), tgt_x in zip(source_loader, target_loader):
            src_x, src_y = src_x.to(device), src_y.to(device)
            tgt_x = tgt_x.to(device)

            cls_out_src, dom_out_src = model(src_x)
            _, dom_out_tgt = model(tgt_x)

            loss_cls = criterion_cls(cls_out_src, src_y)

            # 域分类损失（DANN）
            dom_label_src = torch.zeros(src_x.size(0), dtype=torch.long, device=device)
            dom_label_tgt = torch.ones(tgt_x.size(0), dtype=torch.long, device=device)
            bs_src, bs_tgt = src_x.size(0), tgt_x.size(0)
            loss_dom_src = criterion_domain(dom_out_src, dom_label_src)
            loss_dom_tgt = criterion_domain(dom_out_tgt, dom_label_tgt)

            # 样本数加权的“单个域损失均值”
            loss_dom = (loss_dom_src * bs_src + loss_dom_tgt * bs_tgt) / (bs_src + bs_tgt)

            dom_preds_src = torch.argmax(dom_out_src, dim=1)
            dom_preds_tgt = torch.argmax(dom_out_tgt, dim=1)
            dom_correct += (dom_preds_src == dom_label_src).sum().item()
            dom_correct += (dom_preds_tgt == dom_label_tgt).sum().item()
            dom_total += dom_label_src.size(0) + dom_label_tgt.size(0)
            loss = loss_cls + lambda_ * loss_dom

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            cls_loss_sum += loss_cls.item() * src_x.size(0)
            dom_loss_sum += loss_dom.item() * (src_x.size(0) + tgt_x.size(0))
            total_loss_sum += loss.item() * (src_x.size(0) + tgt_x.size(0))

            total_cls_samples += src_x.size(0)
            total_dom_samples += (src_x.size(0) + tgt_x.size(0))

        avg_cls_loss = cls_loss_sum / total_cls_samples
        avg_dom_loss = dom_loss_sum / total_dom_samples
        avg_total_loss = total_loss_sum / total_dom_samples

        # 域分类准确率（整轮）
        dom_acc = dom_correct / dom_total
        gap = abs(dom_acc - 0.5)

        if scheduler is not None:
            scheduler.step()

        print(f"[Epoch {epoch + 1}] Total Loss: {avg_total_loss:.4f} | "
              f"Cls loss: {avg_cls_loss:.4f} | Dom loss: {avg_dom_loss:.4f} | "
              f"DomAcc: {dom_acc:.4f} | lambda: {lambda_:.4f} ")


        if gap < 0.02 and avg_cls_loss < 0.05 and epoch > 10:
            patience +=1
            if gap < best_gap:
                best_gap = gap
                best_model_state = copy.deepcopy(model.state_dict())
            print(f"[INFO] patience {patience} / 3")
            if patience > 3:
                model.load_state_dict(best_model_state)
                print("[INFO] Early stopping: domain aligned and classifier converged.")
                break
        else:
            patience = 0
            best_gap = gap



    if best_model_state is not None:
        model.load_state_dict(best_model_state)


    return model

In [13]:
if __name__ == '__main__':
    set_seed(seed=11)
    with open("../configs/default.yaml", 'r') as f:
        config = yaml.safe_load(f)['baseline']
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    weight_decay = config['weight_decay']
    num_layers = config['num_layers']
    kernel_size = config['kernel_size']
    start_channels = config['start_channels']
    num_epochs = config['num_epochs']

    source_path = '../datasets/source/train/DC_T197_RP.txt'
    target_path = '../datasets/target/train/HC_T185_RP.txt'
    target_test_path = '../datasets/target/test/HC_T185_RP.txt'
    out_path = 'model'
    os.makedirs(out_path, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Flexible_DANN(num_layers=num_layers,
                          start_channels=start_channels,
                          kernel_size=kernel_size,
                          cnn_act='leakrelu',
                          num_classes=10,
                          lambda_=1).to(device)

    source_loader, target_loader = get_dataloaders(source_path, target_path, batch_size)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs, eta_min=learning_rate * 0.1
    )
    criterion_cls = nn.CrossEntropyLoss()
    criterion_domain = nn.CrossEntropyLoss()

    print("[INFO] Starting standard DANN training (no pseudo labels)...")
    model=train_dann(model, source_loader, target_loader,
               optimizer, criterion_cls, criterion_domain,
               device, num_epochs=40, lambda_=0.5,scheduler=scheduler)

    print("[INFO] Evaluating on target test set...")
    test_dataset = PKLDataset(target_test_path)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    general_test_model(model, criterion_cls, test_loader, device)

[INFO] Starting standard DANN training (no pseudo labels)...
[Epoch 1] Total Loss: 0.8579 | Cls loss: 0.5585 | Dom loss: 0.5972 | DomAcc: 0.6460 | lambda: 0.5000 
[Epoch 2] Total Loss: 0.4123 | Cls loss: 0.0911 | Dom loss: 0.6423 | DomAcc: 0.6280 | lambda: 0.5000 
[Epoch 3] Total Loss: 0.4197 | Cls loss: 0.0620 | Dom loss: 0.7152 | DomAcc: 0.4921 | lambda: 0.5000 
[Epoch 4] Total Loss: 0.3849 | Cls loss: 0.0459 | Dom loss: 0.6780 | DomAcc: 0.5750 | lambda: 0.5000 
[Epoch 5] Total Loss: 0.3670 | Cls loss: 0.0475 | Dom loss: 0.6389 | DomAcc: 0.6440 | lambda: 0.5000 
[Epoch 6] Total Loss: 0.3921 | Cls loss: 0.0378 | Dom loss: 0.7086 | DomAcc: 0.4914 | lambda: 0.5000 
[Epoch 7] Total Loss: 0.3832 | Cls loss: 0.0342 | Dom loss: 0.6979 | DomAcc: 0.5006 | lambda: 0.5000 
[Epoch 8] Total Loss: 0.3699 | Cls loss: 0.0292 | Dom loss: 0.6815 | DomAcc: 0.5693 | lambda: 0.5000 
[Epoch 9] Total Loss: 0.3684 | Cls loss: 0.0217 | Dom loss: 0.6933 | DomAcc: 0.5622 | lambda: 0.5000 
[Epoch 10] Total Loss

In [14]:
if __name__ == '__main__':
    set_seed(seed=22)
    with open("../configs/default.yaml", 'r') as f:
        config = yaml.safe_load(f)['baseline']
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    weight_decay = config['weight_decay']
    num_layers = config['num_layers']
    kernel_size = config['kernel_size']
    start_channels = config['start_channels']
    num_epochs = config['num_epochs']

    source_path = '../datasets/source/train/DC_T197_RP.txt'
    target_path = '../datasets/target/train/HC_T188_RP.txt'
    target_test_path = '../datasets/target/test/HC_T188_RP.txt'
    out_path = 'model'
    os.makedirs(out_path, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Flexible_DANN(num_layers=num_layers,
                          start_channels=start_channels,
                          kernel_size=kernel_size,
                          cnn_act='leakrelu',
                          num_classes=10,
                          lambda_=1).to(device)

    source_loader, target_loader = get_dataloaders(source_path, target_path, batch_size)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs, eta_min=learning_rate * 0.1
    )
    criterion_cls = nn.CrossEntropyLoss()
    criterion_domain = nn.CrossEntropyLoss()

    print("[INFO] Starting standard DANN training (no pseudo labels)...")
    model=train_dann(model, source_loader, target_loader,
               optimizer, criterion_cls, criterion_domain,
               device, num_epochs=40, lambda_=0.5,scheduler=scheduler)

    print("[INFO] Evaluating on target test set...")
    test_dataset = PKLDataset(target_test_path)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    general_test_model(model, criterion_cls, test_loader, device)

[INFO] Starting standard DANN training (no pseudo labels)...
[Epoch 1] Total Loss: 0.8262 | Cls loss: 0.5662 | Dom loss: 0.5189 | DomAcc: 0.7436 | lambda: 0.5000 
[Epoch 2] Total Loss: 0.4059 | Cls loss: 0.1190 | Dom loss: 0.5735 | DomAcc: 0.7017 | lambda: 0.5000 
[Epoch 3] Total Loss: 0.4176 | Cls loss: 0.0839 | Dom loss: 0.6672 | DomAcc: 0.5828 | lambda: 0.5000 
[Epoch 4] Total Loss: 0.3758 | Cls loss: 0.0759 | Dom loss: 0.5997 | DomAcc: 0.6962 | lambda: 0.5000 
[Epoch 5] Total Loss: 0.3868 | Cls loss: 0.0628 | Dom loss: 0.6479 | DomAcc: 0.6069 | lambda: 0.5000 
[Epoch 6] Total Loss: 0.3733 | Cls loss: 0.0599 | Dom loss: 0.6266 | DomAcc: 0.6647 | lambda: 0.5000 
[Epoch 7] Total Loss: 0.3824 | Cls loss: 0.0580 | Dom loss: 0.6489 | DomAcc: 0.6142 | lambda: 0.5000 
[Epoch 8] Total Loss: 0.3537 | Cls loss: 0.0334 | Dom loss: 0.6406 | DomAcc: 0.6555 | lambda: 0.5000 
[Epoch 9] Total Loss: 0.3694 | Cls loss: 0.0384 | Dom loss: 0.6620 | DomAcc: 0.6072 | lambda: 0.5000 
[Epoch 10] Total Loss

In [20]:
if __name__ == '__main__':
    set_seed(seed=133)
    with open("../configs/default.yaml", 'r') as f:
        config = yaml.safe_load(f)['baseline']
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    weight_decay = config['weight_decay']
    num_layers = config['num_layers']
    kernel_size = config['kernel_size']
    start_channels = config['start_channels']
    num_epochs = config['num_epochs']

    source_path = '../datasets/source/train/DC_T197_RP.txt'
    target_path = '../datasets/target/train/HC_T191_RP.txt'
    target_test_path = '../datasets/target/test/HC_T191_RP.txt'
    out_path = 'model'
    os.makedirs(out_path, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Flexible_DANN(num_layers=num_layers,
                          start_channels=start_channels,
                          kernel_size=kernel_size,
                          cnn_act='leakrelu',
                          num_classes=10,
                          lambda_=1).to(device)

    source_loader, target_loader = get_dataloaders(source_path, target_path, batch_size)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs, eta_min=learning_rate * 0.1
    )
    criterion_cls = nn.CrossEntropyLoss()
    criterion_domain = nn.CrossEntropyLoss()

    print("[INFO] Starting standard DANN training (no pseudo labels)...")
    model=train_dann(model, source_loader, target_loader,
               optimizer, criterion_cls, criterion_domain,
               device, num_epochs=30, lambda_=0.5,scheduler=scheduler)

    print("[INFO] Evaluating on target test set...")
    test_dataset = PKLDataset(target_test_path)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    general_test_model(model, criterion_cls, test_loader, device)

[INFO] Starting standard DANN training (no pseudo labels)...
[Epoch 1] Total Loss: 0.7411 | Cls loss: 0.4746 | Dom loss: 0.5333 | DomAcc: 0.7114 | lambda: 0.5000 
[Epoch 2] Total Loss: 0.4195 | Cls loss: 0.1065 | Dom loss: 0.6259 | DomAcc: 0.6285 | lambda: 0.5000 
[Epoch 3] Total Loss: 0.3800 | Cls loss: 0.0757 | Dom loss: 0.6086 | DomAcc: 0.6611 | lambda: 0.5000 
[Epoch 4] Total Loss: 0.3714 | Cls loss: 0.0760 | Dom loss: 0.5908 | DomAcc: 0.6776 | lambda: 0.5000 
[Epoch 5] Total Loss: 0.3920 | Cls loss: 0.0665 | Dom loss: 0.6509 | DomAcc: 0.6052 | lambda: 0.5000 
[Epoch 6] Total Loss: 0.3752 | Cls loss: 0.0616 | Dom loss: 0.6273 | DomAcc: 0.6384 | lambda: 0.5000 
[Epoch 7] Total Loss: 0.3786 | Cls loss: 0.0605 | Dom loss: 0.6362 | DomAcc: 0.6330 | lambda: 0.5000 
[Epoch 8] Total Loss: 0.3672 | Cls loss: 0.0486 | Dom loss: 0.6372 | DomAcc: 0.6372 | lambda: 0.5000 
[Epoch 9] Total Loss: 0.3695 | Cls loss: 0.0416 | Dom loss: 0.6559 | DomAcc: 0.6132 | lambda: 0.5000 
[Epoch 10] Total Loss

In [22]:
if __name__ == '__main__':
    set_seed(seed=133)
    with open("../configs/default.yaml", 'r') as f:
        config = yaml.safe_load(f)['baseline']
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    weight_decay = config['weight_decay']
    num_layers = config['num_layers']
    kernel_size = config['kernel_size']
    start_channels = config['start_channels']
    num_epochs = config['num_epochs']

    source_path = '../datasets/source/train/DC_T197_RP.txt'
    target_path = '../datasets/target/train/HC_T185_RP.txt'
    target_test_path = '../datasets/target/test/HC_T185_RP.txt'
    out_path = 'model'
    os.makedirs(out_path, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Flexible_DANN(num_layers=num_layers,
                          start_channels=start_channels,
                          kernel_size=kernel_size,
                          cnn_act='leakrelu',
                          num_classes=10,
                          lambda_=1).to(device)

    source_loader, target_loader = get_dataloaders(source_path, target_path, batch_size)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs, eta_min=learning_rate * 0.1
    )
    criterion_cls = nn.CrossEntropyLoss()
    criterion_domain = nn.CrossEntropyLoss()

    print("[INFO] Starting standard DANN training (no pseudo labels)...")
    model=train_dann(model, source_loader, target_loader,
               optimizer, criterion_cls, criterion_domain,
               device, num_epochs=20, lambda_=0.5,scheduler=scheduler)

    print("[INFO] Evaluating on target test set...")
    test_dataset = PKLDataset(target_test_path)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    general_test_model(model, criterion_cls, test_loader, device)

[INFO] Starting standard DANN training (no pseudo labels)...
[Epoch 1] Total Loss: 0.7650 | Cls loss: 0.4737 | Dom loss: 0.5823 | DomAcc: 0.6758 | lambda: 0.5000 
[Epoch 2] Total Loss: 0.4321 | Cls loss: 0.1016 | Dom loss: 0.6618 | DomAcc: 0.5734 | lambda: 0.5000 
[Epoch 3] Total Loss: 0.4005 | Cls loss: 0.0762 | Dom loss: 0.6486 | DomAcc: 0.6308 | lambda: 0.5000 
[Epoch 4] Total Loss: 0.4001 | Cls loss: 0.0567 | Dom loss: 0.6866 | DomAcc: 0.5371 | lambda: 0.5000 
[Epoch 5] Total Loss: 0.3956 | Cls loss: 0.0606 | Dom loss: 0.6697 | DomAcc: 0.5776 | lambda: 0.5000 
[Epoch 6] Total Loss: 0.4018 | Cls loss: 0.0556 | Dom loss: 0.6923 | DomAcc: 0.5733 | lambda: 0.5000 
[Epoch 7] Total Loss: 0.3834 | Cls loss: 0.0497 | Dom loss: 0.6674 | DomAcc: 0.6029 | lambda: 0.5000 
[Epoch 8] Total Loss: 0.3761 | Cls loss: 0.0389 | Dom loss: 0.6744 | DomAcc: 0.5786 | lambda: 0.5000 
[Epoch 9] Total Loss: 0.3569 | Cls loss: 0.0320 | Dom loss: 0.6497 | DomAcc: 0.6028 | lambda: 0.5000 
[Epoch 10] Total Loss